# Max Clique problem
Max clique problem is a classic NP-hard problem in Graph theory. The goal is to find the largest clique (a subset of vertices that form a complete subgraph) in a given undirected graph. <br>
This problem has applications in various fields, including:
* **Social network analysis**: Idnetifying highly connected communities.
* **Financial markets**: Detecting highly correlated asset groups for risk assessment and portfolio diversification.
* **Bioinformatics**: Detecting modules in protein-protein interaction networks.

## Problem Definition
Given a undirected graph **G=(V, E)** , where **V** is the set of vertices, **E** is the set of edges, representing connections between vertices. <br>
The objective is to find the largest clique **C** $\subseteq$ **V**, such that for any two vertices **u, v** $\in$ **C**, the edge (u, v) exists in **E**.

### Max Clique graph demo
![](./max_clique_demo.png)

## Modified version: Weighted Max Clique problem
## Problem Definition
Given an undirected graph **G=(V, E)** , where **V** is the set of vertices and **E** is the set of edges representing connections between vertices, <br>
each vertex **v** $\in$ **V** is assigned a non-negative weight **w(v)**.
The objective is to find a clique **C** $\subseteq$ **V** meaning that for any two vertices **u, v** $\in$ **C**, <br>
the edge (u, v) exists in **E** such that the sum of vertex weights $\sum_{v\in C} w(v)$ is maximized.

### Weighted Max Clique demo
![](./max_weighted_clique_graph_demo.png)

## QUBO Hamiltonian for Max clique problem
Because we want to solve this problem by QAOA(Quantum Approximate Optimization Algorithm). Let's see how to redefine this problem into QUBO form. <br>
### Max Clique:
> $H = -\alpha \sum_{i=1}^{n} x_i + \beta \sum_{\substack{(i,j)\notin E \\ i<j}} x_i x_j$ <br>

$x_i$: Chosen vertices. <br> 
$\alpha, \beta$: Parameters that helps algorithm to optimize.
* $-\alpha \sum_{i=1}^{n} x_i$: This term asks QAOA to maximize the number of vertices as much as possible. Negative sign is because We want to minimize this Hamiltonian. <br>
    $\alpha>0$ is the **reward** that encourage algorithm put more vertices inside as clique candidate. 
* $\beta \sum_{\substack{(i,j)\notin E \\ i<j}} x_i x_j$: As the definition of max clique problem, we can't include as much as vertices that's not a clique. <br>
    We have to have a **punishment** that tells the algorithm not to put twos vertices not connected each other. <br>
    In this sence, $\beta>0$ is the **punishment** parameter.

### Weighted Max Clique: 
> $H_{\text{WMC}} \;=\; -\sum_{i=1}^n w_i\,x_i \;+\; \lambda \sum_{\substack{(i,j)\notin E \\ i<j}} x_i x_j$ <br>
- $\sum_{i} w_i x_i$ represents the total weight of chosen vertices.
- A negative sign on $\sum w_i x_i$ rewards choosing vertices with higher weights.
- $\lambda$ penalizes any pair of chosen vertices that are not connected.

**Parameter tuning**: If $\lambda$ is too small, the solution may choose conflicting high-weight vertices; if $\lambda$ is too large, the solver may under-select vertices.  

**Usage**: Once defined, $H$ or $H_{\text{WMC}}$ can be minimized by quantum annealers (e.g., D-Wave), QAOA on gate-based quantum computers, or classical QUBO solvers. The final set $\{i : x_i^* = 1\}$ forms the (weighted) maximum clique under these constraints.

## Max Clique:

$$
H_{\text{MC}} \;=\; -\alpha \sum_{i=1}^n x_i \;+\; \beta \sum_{\substack{(i,j)\notin E \\ i<j}} x_i\, x_j
$$

- **\(x_i\)**: Chosen vertices.  
- **\(\alpha, \beta\)**: Parameters that help the algorithm optimize.

1. \(-\alpha \sum_{i=1}^n x_i\):  
   This term rewards including more vertices in the solution. We use a negative sign because we want to **minimize** the Hamiltonian, so a larger \(\sum x_i\) lowers the overall energy.

2. \(\beta \sum_{\substack{(i,j)\notin E \\ i<j}} x_i x_j\):  
   As a clique requires every pair of chosen vertices to be adjacent, we penalize pairs \((i,j)\) not in \(E\). If both \(x_i\) and \(x_j\) are 1 but \((i,j)\notin E\), this increases the energy. Thus \(\beta > 0\) is a **penalty** factor discouraging invalid pairs.

---

## Weighted Max Clique:

$$
H_{\text{WMC}} 
\;=\;
-\alpha \sum_{i=1}^n w_i\, x_i 
\;+\;
\lambda \sum_{\substack{(i,j)\notin E \\ i<j}} x_i\, x_j
$$

- **\(w_i\)**: The weight of vertex \(i\).  
- **\(x_i\)**: Indicates whether vertex \(i\) is chosen.  
- **\(\alpha, \lambda > 0\)**: Parameters controlling the reward for high‐weight vertices and penalty for non‐adjacent selections.

1. \(-\alpha \sum_{i=1}^n w_i x_i\):  
   This **weighted reward** term encourages choosing vertices with higher weights. Because we are **minimizing** \(H_{\text{WMC}}\), the negative sign helps increase the total weight of selected vertices as much as possible. The parameter \(\alpha\) determines how strongly we value the weights relative to the penalty term.

2. \(\lambda \sum_{\substack{(i,j)\notin E \\ i<j}} x_i x_j\):  
   This **penalty** term ensures that any pair of vertices \((i,j)\) not connected by an edge will raise the energy if both are selected. Thus, \(\lambda\) must be large enough to prevent choosing non‐adjacent vertices simultaneously, preserving the clique property.